In [40]:
import os
os.environ['GRB_LICENSE_FILE'] = r'C:\Users\AKSHAY\gurobi.lic'


In [41]:
import gurobipy as gp 
from gurobipy import GRB


In [42]:
n=int(input())

crystals=[]
for _ in range(n):
    a,b,c=map(int,input().split())
    crystals.append((a,b,c))

for row in crystals:
    print(row)

 4
 10 20 100
 40 50 150
 25 35 200
 50 50 300


(10, 20, 100)
(40, 50, 150)
(25, 35, 200)
(50, 50, 300)


In [35]:
m=int(input())

mines=[]
for _ in range(m):
    a,b,c=map(int,input().split())
    mines.append((a,b,c))

for row in mines:
    print(row)

 3
 15 25 50
 70 60 75
 30 200 60


(15, 25, 50)
(70, 60, 75)
(30, 200, 60)


In [31]:

# Create Gurobi model
model = gp.Model("PolygonOptimizer")

# Define polygon vertices with axis-aligned constraints
num_vertices = 1000
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")
edge_type = model.addVars(num_vertices, vtype=GRB.BINARY, name="edge_type")  # 0=vertical, 1=horizontal

# Axis-alignment constraints
for i in range(num_vertices):
    next_i = (i + 1) % num_vertices
    model.addConstr((edge_type[i] == 1) >> (x[i] == x[next_i]))  # Horizontal edge
    model.addConstr((edge_type[i] == 0) >> (y[i] == y[next_i]))  # Vertical edge

# Closed polygon constraint
model.addConstr(x[0] == x[num_vertices - 1])
model.addConstr(y[0] == y[num_vertices - 1])

# Simplified point inclusion using vertical edges only
def create_point_constraints(px, py, coeff):
    cross_count = 0
    for i in range(num_vertices):
        x1, y1 = x[i], y[i]
        x2, y2 = x[(i + 1) % num_vertices], y[(i + 1) % num_vertices]

        # Only consider vertical edges (edge_type == 0)
        is_vertical = model.addVar(vtype=GRB.BINARY, name=f"is_vertical_{i}")
        model.addConstr(is_vertical == 1 - edge_type[i])  # 1 if vertical, 0 if horizontal

        # Create crossing indicator
        cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")

        # Define binary conditions
        x_cond = model.addVar(vtype=GRB.BINARY, name=f"x_cond_{i}")
        y1_cond = model.addVar(vtype=GRB.BINARY, name=f"y1_cond_{i}")
        y2_cond = model.addVar(vtype=GRB.BINARY, name=f"y2_cond_{i}")

        # Big-M constraints to enforce conditions
        M = 10000  # Large enough constant
        model.addConstr(x1 - px >= -M * (1 - x_cond))  # x1 >= px -> x_cond = 1
        model.addConstr(y2 - py >= -M * (1 - y2_cond))  # y2 >= py -> y2_cond = 1
        model.addConstr(y1 - py <= M * (1 - y1_cond))  # y1 <= py -> y1_cond = 1

        # Ensure cross is 1 only if all conditions hold
        model.addConstr(cross <= is_vertical)
        model.addConstr(cross <= x_cond)
        model.addConstr(cross <= y1_cond)
        model.addConstr(cross <= y2_cond)
        model.addConstr(cross >= is_vertical + x_cond + y1_cond + y2_cond - 3)

        cross_count += cross

    # Odd number of crossings = inside
    inside = model.addVar(vtype=GRB.BINARY, name="inside")

    # Handle modulo using binary constraints
    even_cross = model.addVar(vtype=GRB.BINARY, name="even_cross")
    model.addConstr(cross_count == 2 * even_cross + inside)  # Ensures modulo behavior

    return inside * coeff

# Build objective function
total_score = 0


# Add crystals with positive weight
for a, b, c in crystals:
    total_score += create_point_constraints(a, b, c)

# Add mines with negative weight
for a, b, c in mines:
    total_score -= create_point_constraints(a, b, c)

# Solve parameters
model.setObjective(total_score, GRB.MAXIMIZE)
model.Params.NonConvex = 2
model.Params.TimeLimit = 300  # 5-minute time limit

# Solve and output
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    # Output first 10 vertices for verification
    for i in range(1000):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No valid solution found")


Set parameter NonConvex to value 2
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300
NonConvex  2

Optimize a model with 63009 rows, 38014 columns and 154018 nonzeros
Model fingerprint: 0x6226d544
Model has 2000 simple general constraints
  2000 INDICATOR
Variable types: 2000 continuous, 36014 integer (36014 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [5e+01, 3e+02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+04]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 35009 rows and 28003 columns
Presolve time: 1.13s
Presolved: 28000 rows, 10011 columns, 96986 nonzeros
Variable types: 1998 continuous, 8013 integer (8013 binary)
Found heuristic solution: objective -

In [85]:
#the same above but with the constraints

# Create Gurobi model
model = gp.Model("PolygonOptimizer")

# Define polygon vertices with axis-aligned constraints
num_vertices = 1000
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")
edge_type = model.addVars(num_vertices, vtype=GRB.BINARY, name="edge_type")  # 0=vertical, 1=horizontal

# Axis-alignment constraints
for i in range(num_vertices):
    next_i = (i + 1) % num_vertices
    model.addConstr((edge_type[i] == 1) >> (x[i] == x[next_i]))  # Horizontal edge
    model.addConstr((edge_type[i] == 0) >> (y[i] == y[next_i]))  # Vertical edge

# Closed polygon constraint
model.addConstr(x[0] == x[num_vertices - 1])
model.addConstr(y[0] == y[num_vertices - 1])

cnt = 0
for i in range(1, 998):
    for j in range(i + 2, 1000):
        ximi, xi = x[i - 1], x[i]
        xjmi, xj = x[j - 1], x[j]
        yimi, yi = y[i - 1], y[i]
        yjmi, yj = y[j - 1], y[j]

        num_x = (yjmi - yimi) * (xi - ximi) - (xjmi - ximi) * (yi - yimi)
        den_x = (xj - xjmi) * (yi - yimi) - (xi - ximi) * (yj - yjmi)

        # Define an auxiliary variable for |den_x|
        den_x_abs = model.addVar(lb=0, name=f"den_x_abs_{i}_{j}")
        
        # Add constraints to enforce den_x_abs = |den_x|
        model.addConstr(den_x_abs >= den_x, name=f"abs_pos_{i}_{j}")
        model.addConstr(den_x_abs >= -den_x, name=f"abs_neg_{i}_{j}")
        
        # Instead of if abs(den_x) >= epsilon, use a constraint:
        model.addConstr(den_x_abs >= epsilon, name=f"den_x_nonzero_{i}_{j}")
        
        # Now, we can safely add constraints without checking den_x != 0
        b_x = model.addVar(vtype=GRB.BINARY, name=f"b_x_{i}_{j}")
        model.addConstr(num_x <= -epsilon * den_x + M * b_x, name=f"x_lt_0_{i}_{j}")
        model.addConstr(num_x >= (1 + epsilon) * den_x - M * (1 - b_x), name=f"x_gt_1_{i}_{j}")


        cnt += 1  # Keep track of used variables

        num_y = (yjmi - yimi) * (xj - xjmi) - (xjmi - ximi) * (yj - yjmi)
        den_y = den_x  # Reuse den_x to avoid redundant calculations

        # Define an auxiliary variable for |den_y|
        den_y_abs = model.addVar(lb=0, name=f"den_y_abs_{i}_{j}")
        
        # Add constraints to enforce den_y_abs = |den_y|
        model.addConstr(den_y_abs >= den_y, name=f"abs_pos_{i}_{j}")
        model.addConstr(den_y_abs >= -den_y, name=f"abs_neg_{i}_{j}")
        
        # Instead of if abs(den_y) >= epsilon, use a constraint:
        model.addConstr(den_y_abs >= epsilon, name=f"den_y_nonzero_{i}_{j}")
        
        # Now, we can safely add constraints without checking den_y != 0
        b_y = model.addVar(vtype=GRB.BINARY, name=f"b_y_{i}_{j}")
        model.addConstr(num_y <= -epsilon * den_y + M * b_y, name=f"y_lt_0_{i}_{j}")
        model.addConstr(num_y >= (1 + epsilon) * den_y - M * (1 - b_y), name=f"y_gt_1_{i}_{j}")


        cnt += 1

model.update()


# Simplified point inclusion using vertical edges only
def create_point_constraints(px, py, coeff):
    cross_count = 0
    for i in range(num_vertices):
        x1, y1 = x[i], y[i]
        x2, y2 = x[(i + 1) % num_vertices], y[(i + 1) % num_vertices]

        # Only consider vertical edges (edge_type == 0)
        is_vertical = model.addVar(vtype=GRB.BINARY, name=f"is_vertical_{i}")
        model.addConstr(is_vertical == 1 - edge_type[i])  # 1 if vertical, 0 if horizontal

        # Create crossing indicator
        cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")

        # Define binary conditions
        x_cond = model.addVar(vtype=GRB.BINARY, name=f"x_cond_{i}")
        y1_cond = model.addVar(vtype=GRB.BINARY, name=f"y1_cond_{i}")
        y2_cond = model.addVar(vtype=GRB.BINARY, name=f"y2_cond_{i}")

        # Big-M constraints to enforce conditions
        M = 10000  # Large enough constant
        model.addConstr(x1 - px >= -M * (1 - x_cond))  # x1 >= px -> x_cond = 1
        model.addConstr(y2 - py >= -M * (1 - y2_cond))  # y2 >= py -> y2_cond = 1
        model.addConstr(y1 - py <= M * (1 - y1_cond))  # y1 <= py -> y1_cond = 1

        # Ensure cross is 1 only if all conditions hold
        model.addConstr(cross <= is_vertical)
        model.addConstr(cross <= x_cond)
        model.addConstr(cross <= y1_cond)
        model.addConstr(cross <= y2_cond)
        model.addConstr(cross >= is_vertical + x_cond + y1_cond + y2_cond - 3)

        cross_count += cross

    # Odd number of crossings = inside
    inside = model.addVar(vtype=GRB.BINARY, name="inside")

    # Handle modulo using binary constraints
    even_cross = model.addVar(vtype=GRB.BINARY, name="even_cross")
    model.addConstr(cross_count == 2 * even_cross + inside)  # Ensures modulo behavior

    return inside * coeff

# Build objective function
total_score = 0


# Add crystals with positive weight
for a, b, c in crystals:
    total_score += create_point_constraints(a, b, c)

# Add mines with negative weight
for a, b, c in mines:
    total_score -= create_point_constraints(a, b, c)

# Solve parameters
model.setObjective(total_score, GRB.MAXIMIZE)
model.Params.NonConvex = 2
model.Params.TimeLimit = 300  # 5-minute time limit

# Solve and output
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    # Output first 10 vertices for verification
    for i in range(1000):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No valid solution found")


Set parameter NonConvex to value 2
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300
NonConvex  2

Optimize a model with 1031012 rows, 2013020 columns and 1083018 nonzeros
Model fingerprint: 0x779110f3
Model has 3980024 quadratic constraints
Model has 2000 simple general constraints
  2000 INDICATOR
Variable types: 997006 continuous, 1016014 integer (1016014 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e-06, 1e+00]
  QLMatrix range   [1e+00, 1e+04]
  Objective range  [4e+00, 1e+01]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e-06, 2e+04]
  QRHS range       [1e+04, 1e+04]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 0 rows and 0 columns (presolve time = 7s)...
Pr

In [60]:
import gurobipy as gp
from gurobipy import GRB

# Create Gurobi model
w_model = gp.Model("PolygonOptimizer")

# Define polygon vertices with axis-aligned constraints
# Define polygon vertices with axis-aligned constraints
num_vertices = 1000
x = w_model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = w_model.addVars(num_vertices, lb=0, ub=10000, name="y")
edge_type = w_model.addVars(num_vertices, vtype=GRB.BINARY, name="edge_type")  # 0=vertical, 1=horizontal

# Axis-alignment constraints
for i in range(num_vertices):
    next_i = (i + 1) % num_vertices
    
    w_model.addConstr(edge_type[i] * (x[i] - x[next_i]) == 0)  # x[i] == x[next_i] if horizontal
    w_model.addConstr((1 - edge_type[i]) * (y[i] - y[next_i]) == 0)  # y[i] == y[next_i] if vertical
    
    # Absolute value constraints using auxiliary variables
    abs_x_diff = w_model.addVar(lb=0, ub=10000, name=f"abs_x_diff_{i}")
    abs_y_diff = w_model.addVar(lb=0, ub=10000, name=f"abs_y_diff_{i}")
    
    w_model.addConstr(abs_x_diff >= x[i] - x[next_i])
    w_model.addConstr(abs_x_diff >= -(x[i] - x[next_i]))
    w_model.addConstr(abs_y_diff >= y[i] - y[next_i])
    w_model.addConstr(abs_y_diff >= -(y[i] - y[next_i]))
    
    # Ensure edges have a minimum length to prevent overlapping
    w_model.addConstr(abs_x_diff + abs_y_diff >= 1)

# Closed polygon constraint
w_model.addConstr(x[0] == x[num_vertices - 1])
w_model.addConstr(y[0] == y[num_vertices - 1])


# Simplified point inclusion using vertical edges only
def create_point_constraints(px, py, coeff):
    cross_count = 0
    for i in range(num_vertices):
        x1, y1 = x[i], y[i]
        x2, y2 = x[(i + 1) % num_vertices], y[(i + 1) % num_vertices]

        # Only consider vertical edges (edge_type == 0)
        is_vertical = w_model.addVar(vtype=GRB.BINARY, name=f"is_vertical_{i}")
        w_model.addConstr(is_vertical == 1 - edge_type[i])  # 1 if vertical, 0 if horizontal

        # Create crossing indicator
        cross = w_model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")

        # Define binary conditions
        x_cond = w_model.addVar(vtype=GRB.BINARY, name=f"x_cond_{i}")
        y1_cond = w_model.addVar(vtype=GRB.BINARY, name=f"y1_cond_{i}")
        y2_cond = w_model.addVar(vtype=GRB.BINARY, name=f"y2_cond_{i}")

        # Big-M constraints to enforce conditions
        M = 10000  # Large enough constant
        w_model.addConstr(x1 - px >= -M * (1 - x_cond))  # x1 >= px -> x_cond = 1
        w_model.addConstr(y2 - py >= -M * (1 - y2_cond))  # y2 >= py -> y2_cond = 1
        w_model.addConstr(y1 - py <= M * (1 - y1_cond))  # y1 <= py -> y1_cond = 1

        # Ensure cross is 1 only if all conditions hold
        w_model.addConstr(cross <= is_vertical)
        w_model.addConstr(cross <= x_cond)
        w_model.addConstr(cross <= y1_cond)
        w_model.addConstr(cross <= y2_cond)
        w_model.addConstr(cross >= is_vertical + x_cond + y1_cond + y2_cond - 3)

        cross_count += cross

    # Odd number of crossings = inside
    inside = w_model.addVar(vtype=GRB.BINARY, name="inside")

    # Handle modulo using binary constraints
    even_cross = w_model.addVar(vtype=GRB.BINARY, name="even_cross")
    w_model.addConstr(cross_count == 2 * even_cross + inside)  # Ensures modulo behavior

    return inside * coeff

# Build objective function
total_score = 0

# Add crystals with positive weight
for a, b, c in crystals:
    total_score += create_point_constraints(a, b, c)

# Add mines with negative weight
for a, b, c in mines:
    total_score -= create_point_constraints(a, b, c)

# Solve parameters
w_model.setObjective(total_score, GRB.MAXIMIZE)
w_model.Params.NonConvex = 2
w_model.Params.TimeLimit = 300  # 5-minute time limit
w_model.Params.IterationLimit = 100000  # Set iteration limit

# Solve and output
w_model.optimize()

if w_model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {w_model.objVal}")
    # Output first 10 vertices for verification
    for i in range(1000):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No valid solution found")


Set parameter NonConvex to value 2
Set parameter TimeLimit to value 300
Set parameter IterationLimit to value 100000
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
IterationLimit  100000
TimeLimit  300
NonConvex  2

Optimize a model with 41006 rows, 25008 columns and 102012 nonzeros
Model fingerprint: 0xb925f1f8
Model has 2000 quadratic constraints
Variable types: 4000 continuous, 21008 integer (21008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+02, 3e+02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+04]
Presolve removed 25009 rows and 18003 columns
Presolve time: 0.12s
Presolved: 25987 rows, 18993 columns, 70970 nonzeros
Presolved model h

In [62]:
import gurobipy as gp
from gurobipy import GRB

# Create Gurobi model
w_model = gp.Model("SpiralPolygon")

# Spiral configuration - 250 loops × 4 vertices = 1000 vertices
num_loops = 250
step_size = 40  # Distance between loops

# Vertex coordinates (x, y)
x = w_model.addVars(1000, lb=0, ub=10000, name="x")
y = w_model.addVars(1000, lb=0, ub=10000, name="y")

# Fixed spiral pattern: Right -> Up -> Left -> Down
current_x, current_y = 5000, 5000  # Start at center
M = 10000  # Large constant for big-M constraints

for i in range(1000):
    loop = i // 4  # Current loop number
    direction = i % 4  # 0:Right, 1:Up, 2:Left, 3:Down
    
    if direction == 0:  # Right (horizontal)
        w_model.addConstr(x[i] == current_x + loop * step_size)
        w_model.addConstr(y[i] == current_y)
    elif direction == 1:  # Up (vertical)
        w_model.addConstr(x[i] == current_x + loop * step_size)
        w_model.addConstr(y[i] == current_y + loop * step_size)
    elif direction == 2:  # Left (horizontal)
        w_model.addConstr(x[i] == current_x - loop * step_size)
        w_model.addConstr(y[i] == current_y + loop * step_size)
    else:  # Down (vertical)
        w_model.addConstr(x[i] == current_x - loop * step_size)
        w_model.addConstr(y[i] == current_y - loop * step_size)

# Close the polygon
w_model.addConstr(x[999] == x[0])
w_model.addConstr(y[999] == y[0])

# Point inclusion using vertical edges only
def add_point(px, py, coeff):
    cross_count = 0
    
    for i in range(1000):
        direction = i % 4
        if direction == 1 or direction == 3:  # Vertical edges (Up/Down)
            x1, y1 = x[i], y[i]
            x2, y2 = x[(i+1)%1000], y[(i+1)%1000]
            
            # Create crossing indicator
            cross = w_model.addVar(vtype=GRB.BINARY)
            
            # Auxiliary binary variables for conditions
            x_cond = w_model.addVar(vtype=GRB.BINARY)
            y1_cond = w_model.addVar(vtype=GRB.BINARY)
            y2_cond = w_model.addVar(vtype=GRB.BINARY)
            
            # Big-M constraints to enforce conditions
            w_model.addConstr(x[i] - px >= -M * (1 - x_cond))
            w_model.addConstr(y2 - py >= -M * (1 - y2_cond))
            w_model.addConstr(y1 - py <= M * (1 - y1_cond))
            
            # Ensure cross is 1 only if all conditions hold
            w_model.addConstr(cross <= x_cond)
            w_model.addConstr(cross <= y1_cond)
            w_model.addConstr(cross <= y2_cond)
            w_model.addConstr(cross >= x_cond + y1_cond + y2_cond - 2)
            
            cross_count += cross

    # Odd crossings = inside
    inside = w_model.addVar(vtype=GRB.BINARY)
    even_cross = w_model.addVar(vtype=GRB.BINARY)
    w_model.addConstr(cross_count == 2 * even_cross + inside)
    
    return inside * coeff

# Build objective
total_score = 0

# Add crystals
for a, b, c in crystals:
    total_score += add_point(a, b, c)

# Subtract mines
for a, b, c in mines:
    total_score -= add_point(a, b, c)

# Solve settings
w_model.setObjective(total_score, GRB.MAXIMIZE)
w_model.Params.NonConvex = 2
w_model.Params.MIPGap = 0.1  # 10% optimality gap
w_model.Params.TimeLimit = 600  # 10-minute limit

# Solve
w_model.optimize()

if w_model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {w_model.objVal}")
    print("Sample vertices:")
    for i in range(0, 1000, 100):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No solution found")


Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  600
MIPGap  0.1
NonConvex  2

Optimize a model with 16006 rows, 10008 columns and 36012 nonzeros
Model fingerprint: 0xe7ec158f
Variable types: 2000 continuous, 8008 integer (8008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+02, 3e+02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [2e+00, 1e+04]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 12 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -
No solu

In [63]:
import gurobipy as gp
from gurobipy import GRB

# Create Gurobi model
model = gp.Model("PolygonOptimizer")

# Define polygon vertices with axis-aligned constraints
num_vertices = 1000
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")
edge_type = model.addVars(num_vertices, vtype=GRB.BINARY, name="edge_type")  # 0=vertical, 1=horizontal

# Axis-alignment constraints
for i in range(num_vertices):
    next_i = (i + 1) % num_vertices
    model.addConstr((edge_type[i] == 1) >> (x[i] == x[next_i]))  # Horizontal edge
    model.addConstr((edge_type[i] == 0) >> (y[i] == y[next_i]))  # Vertical edge

# Closed polygon constraint
model.addConstr(x[0] == x[num_vertices - 1])
model.addConstr(y[0] == y[num_vertices - 1])

# Simplified point inclusion using vertical edges only
def create_point_constraints(px, py, coeff):
    cross_count = 0
    for i in range(num_vertices):
        x1, y1 = x[i], y[i]
        x2, y2 = x[(i + 1) % num_vertices], y[(i + 1) % num_vertices]

        is_vertical = model.addVar(vtype=GRB.BINARY, name=f"is_vertical_{i}")
        model.addConstr(is_vertical == 1 - edge_type[i])

        cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")
        x_cond = model.addVar(vtype=GRB.BINARY, name=f"x_cond_{i}")
        y1_cond = model.addVar(vtype=GRB.BINARY, name=f"y1_cond_{i}")
        y2_cond = model.addVar(vtype=GRB.BINARY, name=f"y2_cond_{i}")

        M = 10000
        model.addConstr(x1 - px >= -M * (1 - x_cond))
        model.addConstr(y2 - py >= -M * (1 - y2_cond))
        model.addConstr(y1 - py <= M * (1 - y1_cond))

        model.addConstr(cross <= is_vertical)
        model.addConstr(cross <= x_cond)
        model.addConstr(cross <= y1_cond)
        model.addConstr(cross <= y2_cond)
        model.addConstr(cross >= is_vertical + x_cond + y1_cond + y2_cond - 3)

        cross_count += cross

    inside = model.addVar(vtype=GRB.BINARY, name="inside")
    even_cross = model.addVar(vtype=GRB.BINARY, name="even_cross")
    model.addConstr(cross_count == 2 * even_cross + inside)
    return inside * coeff

# Build objective function
total_score = 0
for a, b, c in crystals:
    total_score += create_point_constraints(a, b, c)
for a, b, c in mines:
    total_score -= create_point_constraints(a, b, c)

# Add constraints to prevent edge intersections
M = 10000
epsilon = 1e-6
b_varss = model.addVars(998000 * 2, vtype=GRB.BINARY, name="b_intersect")
cnt = 0
for i in range(1, 998):
    for j in range(i + 2, 1000):
        num_x = (y[j - 1] - y[i - 1]) * (x[i] - x[i - 1]) - (x[j - 1] - x[i - 1]) * (y[i] - y[i - 1])
        den_x = (x[j] - x[j - 1]) * (y[i] - y[i - 1]) - (x[i] - x[i - 1]) * (y[j] - y[j - 1])
        model.addConstr(num_x <= -epsilon * den_x + M * b_varss[cnt], name=f"x_lt_0_{i}_{j}")
        model.addConstr(num_x >= (1 + epsilon) * den_x - M * (1 - b_varss[cnt]), name=f"x_gt_1_{i}_{j}")
        cnt += 1

        num_y = (y[j - 1] - y[i - 1]) * (x[j] - x[j - 1]) - (x[j - 1] - x[i - 1]) * (y[j] - y[j - 1])
        den_y = den_x
        model.addConstr(num_y <= -epsilon * den_y + M * b_varss[cnt], name=f"y_lt_0_{i}_{j}")
        model.addConstr(num_y >= (1 + epsilon) * den_y - M * (1 - b_varss[cnt]), name=f"y_gt_1_{i}_{j}")
        cnt += 1

model.setObjective(total_score, GRB.MAXIMIZE)
model.Params.NonConvex = 2
model.Params.TimeLimit = 300
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    for i in range(10):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No valid solution found")


Set parameter NonConvex to value 2
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300
NonConvex  2

Optimize a model with 36006 rows, 2019008 columns and 88012 nonzeros
Model fingerprint: 0x6da8f85f
Model has 1990012 quadratic constraints
Model has 2000 simple general constraints
  2000 INDICATOR
Variable types: 2000 continuous, 2017008 integer (2017008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e-06, 1e+00]
  QLMatrix range   [1e+04, 1e+04]
  Objective range  [1e+02, 3e+02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+04]
  QRHS range       [1e+04, 1e+04]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 2 rows and 1000996 columns (presolve time = 7s)...
Pr

GurobiError: Out of memory

In [65]:

# Create Gurobi model
model = gp.Model("PolygonOptimizer")

# Define polygon vertices with axis-aligned constraints
num_vertices = 1000
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")
edge_type = model.addVars(num_vertices, vtype=GRB.BINARY, name="edge_type")  # 0=vertical, 1=horizontal

# Axis-alignment constraints
for i in range(num_vertices):
    next_i = (i + 1) % num_vertices
    model.addConstr((edge_type[i] == 1) >> (x[i] == x[next_i]))  # Horizontal edge
    model.addConstr((edge_type[i] == 0) >> (y[i] == y[next_i]))  # Vertical edge

# Closed polygon constraint
model.addConstr(x[0] == x[num_vertices - 1])
model.addConstr(y[0] == y[num_vertices - 1])

# Simplified point inclusion using vertical edges only
def create_point_constraints(px, py, coeff):
    cross_count = 0
    for i in range(num_vertices):
        x1, y1 = x[i], y[i]
        x2, y2 = x[(i + 1) % num_vertices], y[(i + 1) % num_vertices]

        # Determine edge orientation
        is_vertical = model.addVar(vtype=GRB.BINARY, name=f"is_vertical_{i}")
        is_horizontal = model.addVar(vtype=GRB.BINARY, name=f"is_horizontal_{i}")
        
        model.addConstr(is_vertical == 1 - edge_type[i])  # 1 if vertical, 0 otherwise
        model.addConstr(is_horizontal == edge_type[i])  # 1 if horizontal, 0 otherwise

        # Create crossing indicator
        cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")

        # Binary conditions for vertical edges
        x_cond = model.addVar(vtype=GRB.BINARY, name=f"x_cond_{i}")
        y1_cond = model.addVar(vtype=GRB.BINARY, name=f"y1_cond_{i}")
        y2_cond = model.addVar(vtype=GRB.BINARY, name=f"y2_cond_{i}")

        # Binary conditions for horizontal edges
        y_cond = model.addVar(vtype=GRB.BINARY, name=f"y_cond_{i}")
        x1_cond = model.addVar(vtype=GRB.BINARY, name=f"x1_cond_{i}")
        x2_cond = model.addVar(vtype=GRB.BINARY, name=f"x2_cond_{i}")
        
        # Big-M constraints
        M = 10000

        # Constraints for vertical edges (checking x-crossing)
        model.addConstr(x1 - px >= -M * (1 - x_cond))
        model.addConstr(y2 - py >= -M * (1 - y2_cond))
        model.addConstr(y1 - py <= M * (1 - y1_cond))
        
        # Constraints for horizontal edges (checking y-crossing)
        model.addConstr(y1 - py >= -M * (1 - y_cond))
        model.addConstr(x2 - px >= -M * (1 - x2_cond))
        model.addConstr(x1 - px <= M * (1 - x1_cond))

        # Auxiliary variables for product terms
        vert_cross = model.addVar(vtype=GRB.BINARY, name=f"vert_cross_{i}")
        horz_cross = model.addVar(vtype=GRB.BINARY, name=f"horz_cross_{i}")
        
        model.addConstr(vert_cross <= x_cond)
        model.addConstr(vert_cross <= y1_cond)
        model.addConstr(vert_cross <= y2_cond)
        model.addConstr(vert_cross >= x_cond + y1_cond + y2_cond - 2)
        
        model.addConstr(horz_cross <= y_cond)
        model.addConstr(horz_cross <= x1_cond)
        model.addConstr(horz_cross <= x2_cond)
        model.addConstr(horz_cross >= y_cond + x1_cond + x2_cond - 2)

        # Ensure cross is 1 if conditions hold
        model.addConstr(cross <= is_vertical * vert_cross + is_horizontal * horz_cross)
        model.addConstr(cross >= is_vertical * vert_cross)
        model.addConstr(cross >= is_horizontal * horz_cross)

        cross_count += cross

    # Odd number of crossings = inside
    inside = model.addVar(vtype=GRB.BINARY, name="inside")

    # Handle modulo using binary constraints
    even_cross = model.addVar(vtype=GRB.BINARY, name="even_cross")
    model.addConstr(cross_count == 2 * even_cross + inside)  # Ensures modulo behavior

    return inside * coeff

# Build objective function
total_score = 0


# Add crystals with positive weight
for a, b, c in crystals:
    total_score += create_point_constraints(a, b, c)

# Add mines with negative weight
for a, b, c in mines:
    total_score -= create_point_constraints(a, b, c)

# Solve parameters
model.setObjective(total_score, GRB.MAXIMIZE)
model.Params.NonConvex = 2
model.Params.TimeLimit = 300  # 5-minute time limit

# Solve and output
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    # Output first 10 vertices for verification
    for i in range(1000):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No valid solution found")


Set parameter NonConvex to value 2
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300
NonConvex  2

Optimize a model with 64006 rows, 47008 columns and 148012 nonzeros
Model fingerprint: 0xe1926251
Model has 12000 quadratic constraints
Model has 2000 simple general constraints
  2000 INDICATOR
Variable types: 2000 continuous, 45008 integer (45008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+02, 3e+02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+04]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 24010 rows and 32002 columns
Presolve time: 0.44s
Presolved: 47996 rows, 19006 columns, 132081

In [66]:
model = gp.Model("SpiralPolygon")

# Spiral configuration - 250 loops × 4 vertices = 1000 vertices
num_loops = 250
step_size = 40  # Distance between loops

# Vertex coordinates (x, y)
x = model.addVars(1000, lb=0, ub=10000, name="x")
y = model.addVars(1000, lb=0, ub=10000, name="y")

# Fixed spiral pattern to prevent intersections
base_x, base_y = 5000, 5000  # Center point
for i in range(1000):
    loop = i // 4  # Current loop number
    direction = i % 4  # 0:Right, 1:Up, 2:Left, 3:Down
    
    if direction == 0:  # Right (horizontal)
        model.addConstr(x[i] == base_x + loop*step_size)
        model.addConstr(y[i] == base_y - loop*step_size)
    elif direction == 1:  # Up (vertical)
        model.addConstr(x[i] == base_x + loop*step_size)
        model.addConstr(y[i] == base_y + loop*step_size)
    elif direction == 2:  # Left (horizontal)
        model.addConstr(x[i] == base_x - loop*step_size)
        model.addConstr(y[i] == base_y + loop*step_size)
    else:  # Down (vertical)
        model.addConstr(x[i] == base_x - loop*step_size)
        model.addConstr(y[i] == base_y - loop*step_size)

# Close the polygon
model.addConstr(x[999] == x[0])
model.addConstr(y[999] == y[0])

# Point inclusion using vertical edges only (ray-casting algorithm)
def add_point(px, py, coeff):
    cross_count = 0
    
    for i in range(1000):
        direction = i % 4
        if direction == 1 or direction == 3:  # Vertical edges (Up/Down)
            x_val = x[i]
            y1 = y[i]
            y2 = y[(i+1)%1000]
            
            # Create crossing indicator
            cross = model.addVar(vtype=GRB.BINARY)
            
            # Crossing conditions
            model.addConstr(cross <= (x_val >= px))
            model.addConstr(cross <= (gp.max_(y1, y2) >= py))
            model.addConstr(cross <= (gp.min_(y1, y2) <= py))
            model.addConstr(cross >= (x_val >= px) + (gp.max_(y1, y2) >= py) + (gp.min_(y1, y2) <= py) - 2)
            
            cross_count += cross

    # Odd crossings = inside
    inside = model.addVar(vtype=GRB.BINARY)
    model.addConstr(inside == cross_count % 2)
    return inside * coeff

# Build objective
total_score = 0

# Add crystals
for a, b, c in crystals:
    total_score += add_point(a, b, c)

# Subtract mines
for a, b, c in mines:
    total_score -= add_point(a, b, c)

# Solve settings
model.setObjective(total_score, GRB.MAXIMIZE)
model.Params.NonConvex = 2
model.Params.MIPGap = 0.1  # 10% optimality gap
model.Params.TimeLimit = 600  # 10-minute limit

# Solve
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    print("First 10 vertices:")
    for i in range(10):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No solution found")

TypeError: unsupported operand type(s) for -: 'gurobipy._core.LinExpr' and 'TempConstr'

In [67]:
import gurobipy as gp
from gurobipy import GRB

# Create model
model = gp.Model("SpiralPolygon")

# Spiral configuration
num_loops = 250
num_vertices = num_loops * 4  # 1000 vertices
dist = 40  # Step size between loops

# Vertex coordinates
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")

# Center point
base_x, base_y = 5000, 5000

# Define spiral structure
for i in range(num_vertices):
    loop = i // 4  # Current loop
    direction = i % 4  # 0: Right, 1: Up, 2: Left, 3: Down
    
    if direction == 0:  # Right
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)
    elif direction == 1:  # Up
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    elif direction == 2:  # Left
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    else:  # Down
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)

# Close the polygon
model.addConstr(x[num_vertices - 1] == x[0])
model.addConstr(y[num_vertices - 1] == y[0])

# Function to determine point inclusion

def add_point(px, py, coeff):
    cross_count = model.addVar(vtype=GRB.INTEGER, name=f"cross_count_{px}_{py}")
    model.addConstr(cross_count == 0)
    
    for i in range(num_vertices):
        direction = i % 4
        if direction == 1 or direction == 3:  # Vertical edges (Up/Down)
            y1 = y[i]
            y2 = y[(i + 1) % num_vertices]
            x_val = x[i]
            
            cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")
            
            model.addConstr(cross <= (x_val >= px).astype(int))
            model.addConstr(cross <= (gp.max_(y1, y2) >= py).astype(int))
            model.addConstr(cross <= (gp.min_(y1, y2) <= py).astype(int))
            
            model.addConstr(cross_count >= cross_count + cross - 1)
    
    inside = model.addVar(vtype=GRB.BINARY, name=f"inside_{px}_{py}")
    model.addConstr(inside == (cross_count % 2))
    return inside * coeff

# Objective function
crystals = [(1000, 2000, 5), (3000, 4000, 10)]  # Example crystal points
mines = [(7000, 8000, 7), (9000, 1000, 4)]  # Example mine points

total_score = gp.LinExpr()
for a, b, c in crystals:
    total_score += add_point(a, b, c)
for a, b, c in mines:
    total_score -= add_point(a, b, c)

model.setObjective(total_score, GRB.MAXIMIZE)

# Solve settings
model.Params.NonConvex = 2
model.Params.MIPGap = 0.1
model.Params.TimeLimit = 600

# Solve the model
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    print("First 10 vertices:")
    for i in range(10):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No solution found")


AttributeError: 'TempConstr' object has no attribute 'astype'

In [68]:
import gurobipy as gp
from gurobipy import GRB

# Create model
model = gp.Model("SpiralPolygon")

# Spiral configuration
num_loops = 250
num_vertices = num_loops * 4  # 1000 vertices
dist = 40  # Step size between loops

# Vertex coordinates
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")

# Center point
base_x, base_y = 5000, 5000

# Define spiral structure
for i in range(num_vertices):
    loop = i // 4  # Current loop
    direction = i % 4  # 0: Right, 1: Up, 2: Left, 3: Down
    
    if direction == 0:  # Right
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)
    elif direction == 1:  # Up
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    elif direction == 2:  # Left
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    else:  # Down
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)

# Close the polygon
model.addConstr(x[num_vertices - 1] == x[0])
model.addConstr(y[num_vertices - 1] == y[0])

# Function to determine point inclusion

def add_point(px, py, coeff):
    cross_count = model.addVar(vtype=GRB.INTEGER, name=f"cross_count_{px}_{py}")
    model.addConstr(cross_count == 0)
    
    for i in range(num_vertices):
        direction = i % 4
        if direction == 1 or direction == 3:  # Vertical edges (Up/Down)
            y1 = y[i]
            y2 = y[(i + 1) % num_vertices]
            x_val = x[i]
            
            cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")
            
            # Implementing the condition manually by introducing binary variables
            cross_condition = model.addVar(vtype=GRB.BINARY)
            model.addConstr(cross_condition == (x_val >= px))  # This will set cross_condition to 1 if x_val >= px
            model.addConstr(cross_condition == 1 if (max(y1, y2) >= py and min(y1, y2) <= py) else 0)
            
            model.addConstr(cross == cross_condition)

            model.addConstr(cross_count >= cross_count + cross - 1)
    
    inside = model.addVar(vtype=GRB.BINARY, name=f"inside_{px}_{py}")
    model.addConstr(inside == (cross_count % 2))
    return inside * coeff



total_score = gp.LinExpr()
for a, b, c in crystals:
    total_score += add_point(a, b, c)
for a, b, c in mines:
    total_score -= add_point(a, b, c)

model.setObjective(total_score, GRB.MAXIMIZE)

# Solve settings
model.Params.NonConvex = 2
model.Params.MIPGap = 0.1
model.Params.TimeLimit = 600

# Solve the model
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    print("First 10 vertices:")
    for i in range(10):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No solution found")


TypeError: unsupported operand type(s) for -: 'gurobipy._core.LinExpr' and 'TempConstr'

In [69]:
import gurobipy as gp
from gurobipy import GRB

# Create model
model = gp.Model("SpiralPolygon")

# Spiral configuration
num_loops = 250
num_vertices = num_loops * 4  # 1000 vertices
dist = 40  # Step size between loops

# Vertex coordinates
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")

# Center point
base_x, base_y = 5000, 5000

# Define spiral structure
for i in range(num_vertices):
    loop = i // 4  # Current loop
    direction = i % 4  # 0: Right, 1: Up, 2: Left, 3: Down
    
    if direction == 0:  # Right
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)
    elif direction == 1:  # Up
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    elif direction == 2:  # Left
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    else:  # Down
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)

# Close the polygon
model.addConstr(x[num_vertices - 1] == x[0])
model.addConstr(y[num_vertices - 1] == y[0])

# Function to determine point inclusion

def add_point(px, py, coeff):
    cross_count = model.addVar(vtype=GRB.INTEGER, name=f"cross_count_{px}_{py}")
    model.addConstr(cross_count == 0)
    
    for i in range(num_vertices):
        direction = i % 4
        if direction == 1 or direction == 3:  # Vertical edges (Up/Down)
            y1 = y[i]
            y2 = y[(i + 1) % num_vertices]
            x_val = x[i]
            
            # Create binary variable for crossing condition
            cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")
            
            # Create conditions for crossing (x_val >= px and y1 <= py <= y2)
            cond1 = model.addConstr(x_val >= px)
            cond2 = model.addConstr(gp.min_(y1, y2) <= py)
            cond3 = model.addConstr(gp.max_(y1, y2) >= py)
            
            # Ensure crossing happens only if all conditions are met
            model.addConstr(cross == (cond1.rhs and cond2.rhs and cond3.rhs))  # Check if point is between y1 and y2
            
            model.addConstr(cross_count >= cross_count + cross - 1)
    
    inside = model.addVar(vtype=GRB.BINARY, name=f"inside_{px}_{py}")
    model.addConstr(inside == (cross_count % 2))
    return inside * coeff

# Objective function
crystals = [(1000, 2000, 5), (3000, 4000, 10)]  # Example crystal points
mines = [(7000, 8000, 7), (9000, 1000, 4)]  # Example mine points

total_score = gp.LinExpr()
for a, b, c in crystals:
    total_score += add_point(a, b, c)
for a, b, c in mines:
    total_score -= add_point(a, b, c)

model.setObjective(total_score, GRB.MAXIMIZE)

# Solve settings
model.Params.NonConvex = 2
model.Params.MIPGap = 0.1
model.Params.TimeLimit = 600

# Solve the model
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    print("First 10 vertices:")
    for i in range(10):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No solution found")


TypeError: '<=' not supported between instances of 'GenExprMin' and 'int'

In [78]:
import gurobipy as gp
from gurobipy import GRB

# Create model
model = gp.Model("SpiralPolygon")

# Spiral configuration
num_loops = 250
num_vertices = num_loops * 4  # 1000 vertices
dist = 40  # Step size between loops

# Vertex coordinates
x = model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = model.addVars(num_vertices, lb=0, ub=10000, name="y")

# Center point
base_x, base_y = 5000, 5000

# Define spiral structure
for i in range(num_vertices):
    loop = i // 4  # Current loop
    direction = i % 4  # 0: Right, 1: Up, 2: Left, 3: Down
    
    if direction == 0:  # Right
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)
    elif direction == 1:  # Up
        model.addConstr(x[i] == base_x + loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    elif direction == 2:  # Left
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y + loop * dist)
    else:  # Down
        model.addConstr(x[i] == base_x - loop * dist)
        model.addConstr(y[i] == base_y - loop * dist)

# Close the polygon
model.addConstr(x[num_vertices - 1] == x[0])
model.addConstr(y[num_vertices - 1] == y[0])

# Function to determine point inclusion
def add_point(px, py, coeff):
    cross_count = model.addVar(vtype=GRB.INTEGER, name=f"cross_count_{px}_{py}")
    model.addConstr(cross_count == 0)
    
    for i in range(num_vertices):
        direction = i % 4
        if direction == 1 or direction == 3:  # Vertical edges (Up/Down)
            y1 = y[i]
            y2 = y[(i + 1) % num_vertices]
            x_val = x[i]
            
            # Binary variable to represent the condition x_val >= px
            cross_x_condition = model.addVar(vtype=GRB.BINARY, name=f"cross_x_condition_{i}")
            
            # Implement the condition x_val >= px using big-M method
            M = 10000  # A large number (could be adjusted)
            model.addConstr(x_val - px >= -M * (1 - cross_x_condition))
            model.addConstr(x_val - px <= M * cross_x_condition)
            
            # Create binary variables for the y-condition (py lies between y1 and y2)
            cross_y_min_condition = model.addVar(vtype=GRB.BINARY, name=f"cross_y_min_condition_{i}")
            model.addConstr(cross_y_min_condition == (y1 - py <= 0))  # y1 <= py
            model.addConstr(y1 - py <= 0)  # Explicit constraint to handle the condition
            
            cross_y_max_condition = model.addVar(vtype=GRB.BINARY, name=f"cross_y_max_condition_{i}")
            model.addConstr(cross_y_max_condition == (y2 - py >= 0))  # y2 >= py
            model.addConstr(y2 - py >= 0)  # Explicit constraint to handle the condition
            
            # Combine both conditions into the cross variable
            cross = model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")
            model.addConstr(cross == cross_x_condition * cross_y_min_condition * cross_y_max_condition)
            
            # Update the cross count
            model.addConstr(cross_count >= cross_count + cross - 1)
    
    # Determine if the point is inside using the crossing number
    inside = model.addVar(vtype=GRB.BINARY, name=f"inside_{px}_{py}")
    model.addConstr(inside == (cross_count % 2))
    
    return inside * coeff


total_score = gp.LinExpr()
for a, b, c in crystals:
    total_score += add_point(a, b, c)
for a, b, c in mines:
    total_score -= add_point(a, b, c)

model.setObjective(total_score, GRB.MAXIMIZE)

# Solve settings
model.Params.NonConvex = 2
model.Params.MIPGap = 0.1
model.Params.TimeLimit = 600

# Solve the model
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
    print("First 10 vertices:")
    for i in range(10):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No solution found")


TypeError: unsupported operand type(s) for -: 'gurobipy._core.LinExpr' and 'TempConstr'